In [218]:
import pandas as pd

#a)

df = pd.read_csv('taxi_trip_data.csv',encoding='unicode_escape')

df = df.iloc[:15000]

df = df.drop(columns=['store_and_fwd_flag', 'rate_code', 'total_amount'])

df.to_csv('taxitripdatanew.csv', index=False)

df2 = pd.read_csv('taxitripdatanew.csv',encoding='unicode_escape')
df3=pd.read_csv('taxi_zone_geo.csv',encoding='unicode_escape')
#b)


essentials= ['fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'imp_surcharge', 'payment_type', 'tip_amount'  ,'pickup_datetime', 'dropoff_datetime']
df2 = df2.dropna(subset=essentials)


In [5]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import csv
from datetime import datetime
import string
cloud_config= {
  'secure_connect_bundle': 'C:/Users/Ace/OneDrive/Desktop/GIU/Semester 6/Big Data/secure-connect-mydb.zip'
}
auth_provider = PlainTextAuthProvider('NMCBhlSFospdgDTGgAySwzGO', 'gyJE7tQ-QQKK75AIL5PzPjEpFKuYB5ep3wRlE_N7Zo,.wTRG,0rCckmUxjsAJRQQv+X2eE3Y1pT_HZj,R.46enjqkofmGZq7516vJY+9wmuJa7.mlqf1YmjZ3fEl.Mns')
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

session.set_keyspace('mykeyspace') # replace <<KEYSPACENAME>> with the name of the keyspace you created in DataStax
row = cluster.metadata.keyspaces['mykeyspace']
#c)
session.execute("""
     CREATE TABLE IF NOT EXISTS mykeyspace.Trip  (
         vendor_id INT,
         pickup_datetime TIMESTAMP,
         dropoff_datetime TIMESTAMP,
         passenger_count INT,
         trip_distance FLOAT,
         payment_type TEXT,
         fare_amount FLOAT,
         extra FLOAT,
         mta_tax FLOAT ,
         tip_amount FLOAT,
         tolls_amount FLOAT,
         imp_surcharge FLOAT,
         pickup_location_id INT ,
         dropoff_location_id INT ,
         duration INT,
         trip_cost FLOAT,
         type_of_day TEXT,
         PRIMARY KEY (pickup_location_id, dropoff_location_id)
  	);
 """)


In [8]:
df = pd.read_csv('taxitripdatanew.csv', encoding="ISO-8859-1")
for x in range(0, 15000):
    session.execute("""
     INSERT INTO mykeyspace.Trip (
          vendor_id,
          pickup_datetime,
          dropoff_datetime,
          passenger_count,
          trip_distance,
          payment_type,
          fare_amount,
          extra,
          mta_tax,
          tip_amount,
          tolls_amount,
          imp_surcharge,
          pickup_location_id,
          dropoff_location_id
     ) VALUES (
          %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,
          %s, %s, %s, %s
     );""",
     (
          int(df.iloc[x].vendor_id),
          str(df.iloc[x].pickup_datetime),
          str(df.iloc[x].dropoff_datetime),
          int(df.iloc[x].passenger_count),
          float(df.iloc[x].trip_distance),
          str(df.iloc[x].payment_type),
          float(df.iloc[x].fare_amount),
          float(df.iloc[x].extra),
          float(df.iloc[x].mta_tax),
          float(df.iloc[x].tip_amount),
          float(df.iloc[x].tolls_amount),
          float(df.iloc[x].imp_surcharge),
          int(df.iloc[x].pickup_location_id),
          int(df.iloc[x].dropoff_location_id)
))

In [219]:
#D)

rows = session.execute('SELECT * FROM Trip')
for row in rows:
    pickup_datetime_str = row.pickup_datetime
    dropoff_datetime_str = row.dropoff_datetime

    # Parse strings into datetime objects
    pickup_datetime = datetime.strptime(pickup_datetime_str, '%Y-%m-%d %H:%M:%S')
    dropoff_datetime = datetime.strptime(dropoff_datetime_str, '%Y-%m-%d %H:%M:%S')

    # Calculate trip duration
    trip_duration = (dropoff_datetime - pickup_datetime).total_seconds()
    
    # Update trip_duration in Cassandra
    session.execute("UPDATE Trip SET trip_duration = %s WHERE pickup_location_id = %s AND dropoff_location_id = %s",
                    (int(trip_duration), row.pickup_location_id, row.dropoff_location_id))


In [118]:
#E)
#session.execute("alter table mykeyspace.Trip add total_cost float;")
rows = session.execute('SELECT * FROM Trip')
count=0.0
for i in rows:
  count = i.fare_amount + i.extra + i.mta_tax + i.tip_amount + i.tolls_amount + i.imp_surcharge
  session.execute("UPDATE mykeyspace.Trip SET total_cost = %s WHERE pickup_location_id= %s AND dropoff_location_id = %s",(count,i.pickup_location_id,i.dropoff_location_id))
  count = 0.0

In [222]:
#G)
# Execute the query to calculate average tip amount per passenger count
rows = session.execute("SELECT passenger_count, AVG(tip_amount) AS avg_tip_amount FROM Trip WHERE passenger_count IN (1, 2, 3, 4, 5, 6) GROUP BY passenger_count")

# Print the average tip amounts for each passenger count
for row in rows:
   print("Count:", row.passenger_count, "Average Tip Amount:", row.avg_tip_amount)


Count:  1  avg  2.414156436920166
Count:  2  avg  2.4003334045410156
Count:  3  avg  2.1173417568206787
Count:  4  avg  4.218235492706299
Count:  5  avg  2.8640875816345215
Count:  6  avg  1.8402061462402344


In [346]:
#F)
M = datetime.strptime('06:00:00', '%H:%M:%S').time().strftime('%H:%M:%S')
A = datetime.strptime('12:00:00', '%H:%M:%S').time().strftime('%H:%M:%S')
E = datetime.strptime('18:00:00', '%H:%M:%S').time().strftime('%H:%M:%S')
Morning_Time=datetime.strptime(M,'%H:%M:%S').hour
AfterNoon_Time=datetime.strptime(A,'%H:%M:%S').hour
Evening_Time=datetime.strptime(E,'%H:%M:%S').hour

# Define the time periods
time_periods = ["Morning", "Afternoon", "Evening", "Night"]

# Query the table for each time period
for period in time_periods:
    rows = session.execute(
        "SELECT payment_type, COUNT(*) AS count FROM Trip WHERE daytime = %s AND payment_type IS NOT NULL GROUP BY payment_type",
        (period,)
    )
    
    # Find the most common payment type
    max_count = 0
    common_payment_type = None
    for row in rows:
        if row.count > max_count:
            max_count = row.count
            common_payment_type = row.payment_type
    
    # Print the result
    print("Common Payment_Type in", period, "is:", common_payment_type, "with count of:", max_count)

0
Common Payment_Type in Morning is: 1 with count of: 553
Common Payment_Type in Evening is: 1 with count of: 125
Common Payment_Type in AfterNoon is: 1 with count of: 609
Common Payment_Type in Night is: 1 with count of: 0


In [224]:
#H)
session.execute("""
    CREATE TABLE IF NOT EXISTS mykeyspace.Geo  (
        zone_id INT,
        zone_name TEXT,
        PRIMARY KEY (zone_id)
    );
""")
df2 = pd.read_csv('taxi_zone_geo.csv', encoding="ISO-8859-1")

for x in range(0,len(df2)):
    session.execute("""
     INSERT INTO mykeyspace.Geo (
          zone_id,
          zone_name
     ) VALUES (
          %s, %s
     );""",
     (
          int(df2.iloc[x].zone_id),
          str(df2.iloc[x].zone_name),
))


In [338]:
# Retrieve the top locations with the highest average fares
rows = session.execute("""
   SELECT Geo.zone_name, AVG(fare_amount) AS avg_fare
   FROM Trip
   JOIN Geo ON Geo.zone_id = Trip.pickup_location_id
   GROUP BY Geo.zone_id
   ORDER BY avg_fare DESC
   LIMIT 5
""")

# Print the top locations and their average fares
ranking = ["first", "second", "third", "fourth", "fifth"]
for i, row in enumerate(rows):
   print("The", ranking[i], "best location to pick up passengers is:", row.zone_name, "with avg fares of:", row.avg_fare)


The first best location to pick up passengers is: Springfield Gardens South with avg fares of: 150.0
The second best location to pick up passengers is: Riverdale/North Riverdale/Fieldston with avg fares of: 79.0
The third best location to pick up passengers is: East New York/Pennsylvania Avenue with avg fares of: 66.5
The fourth best location to pick up passengers is: Coney Island with avg fares of: 61.0
The fifth best location to pick up passengers is: Saint Albans with avg fares of: 58.0
